In [1]:
# this implementation employs ideas from
# https://people.eecs.berkeley.edu/~nagaban2/resources/portfolio/convex_report.pdf


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class PosyNet(nn.Module):

    def __init__(self):
        super(PosyNet, self).__init__()
        
        # input: 3-vector (x, y, z)
        self.len_x = 3
        # output: 1-vector, posynomial approximation
        
        # number of monomials: M
        self.M = 4
        
        
        self.w = nn.Parameter(6*torch.rand(self.len_x, self.M)-3)
        #self.powers = torch.FloatTensor([[0, -1, 1, 1, 1],[1, -2, 0, 2, 1], [0, -1, 0, 2, 1]]).t()
        self.b = nn.Parameter(1.5*torch.rand(self.M))
        
        
    def forward(self, x):
        
        y = torch.log(x)
        
        z = [torch.log(1+torch.exp(torch.matmul(self.w[:,i], y) + self.b[i])) for i in range(self.M)]
        
        zh = [torch.exp(zz) for zz in z]
        
        posy = sum(zh)-self.M
           
        return posy
    


In [4]:
def f(x):

    out = x[1]**1.5*x[2]**3 + 2*x[0]**2*x[2]**-1 + 3*x[1]**3.2 + 4*x[0]**0.5*x[1]**-2*x[2]
    
    return out

In [5]:
net = PosyNet()
print(net)

import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

torch.set_printoptions(precision=3, sci_mode=False)

PosyNet()


In [6]:
list(net.parameters())

[Parameter containing:
 tensor([[-2.665, -2.345, -1.106,  0.561],
         [ 0.737, -1.729,  2.545,  1.360],
         [ 1.624, -1.250, -0.792, -1.107]], requires_grad=True),
 Parameter containing:
 tensor([1.111, 0.989, 0.980, 1.210], requires_grad=True)]

In [11]:
relative_loss_hist

[0.014696675288911745,
 0.014027319868588772,
 0.015416312233671002,
 0.012989991983607372,
 0.013021747747297923,
 0.01244398567425378,
 0.01236179072956368,
 0.013879970162764342,
 0.011869032476575588,
 0.013850560001169753,
 0.01253045438385167,
 0.012435305231665552,
 0.012823629608792544,
 0.012215358497610396,
 0.01231808281792534]